In [1]:
import pickle
from statistics import mean
import numpy as np
import sys

In [2]:
file_name_train_250 = "../Human Experiments/Data/RL_trajectories_1000_250.pickle"
file_obj_train_250 = open(file_name_train_250, 'rb')
trajectories_train_250 = pickle.load(file_obj_train_250)
trajectories_train_250 = [list(ele) for ele in trajectories_train_250]

file_name_test_250 = "../Human Experiments/Data/Human_trajectories_test_250.pickle"
file_obj_test_250 = open(file_name_test_250, 'rb')
trajectories_test_250 = pickle.load(file_obj_test_250)
trajectories_test_250 = [list(ele) for ele in trajectories_test_250]

file_name_train_500 = "../Human Experiments/Data/RL_trajectories_1000_500.pickle"
file_obj_train_500 = open(file_name_train_500, 'rb')
trajectories_train_500 = pickle.load(file_obj_train_500)
trajectories_train_500 = [list(ele) for ele in trajectories_train_500]

file_name_test_500 = "../Human Experiments/Data/Human_trajectories_test_500.pickle"
file_obj_test_500 = open(file_name_test_500, 'rb')
trajectories_test_500 = pickle.load(file_obj_test_500)
trajectories_test_500 = [list(ele) for ele in trajectories_test_500]

test_size = 60

In [3]:
def generate_policies_two_split(price_low_bound, price_upper_bound, step_size, split):
    policies = []
    for i in range(price_low_bound, price_upper_bound, step_size):
        for j in range(price_low_bound, price_upper_bound, step_size):
            for k in range(split[0]):
                for l in range(split[1]):
                    policy = (i, j, k, l)
                    policies.append(policy)
    return policies

In [4]:
policies = generate_policies_two_split(150, 260, 10, (8,7))
len(policies)

6776

In [5]:
def evaluate_policy_two_split(policy, data, best_buys_cost, best_buys_idx, split, get_final_output):
    if(len(policy)!=4):
        raise ValueError("Number of parameters in this policy is "+ str(len(policy))+" when it should be 4")
    bought = []
    bought_cost_only = []
    bought_idx_only = []
    t1 = policy[0]
    t2 = policy[1]
    d1 = policy[2]
    d2 = policy[3]
    for trajectory in data:
        for idx, cost in enumerate(trajectory):
            if (idx<split[0]):
                local_idx = idx
            else:
                local_idx = idx-split[0]
            if(idx == len(trajectory) - 1):
                bought.append((cost, idx))
                bought_cost_only.append(cost)
                bought_idx_only.append(idx)
                break
            elif(idx < split[0]):
                if(cost > t1 and local_idx < d1):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(idx >= split[0]):
                if(cost > t2 and local_idx < d2):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
    error = [a_i - b_i for a_i, b_i in zip(bought_cost_only, best_buys_cost)]
    correct_stops = (np.equal(bought_idx_only, best_buys_idx)).astype(int)
    correct_stops = sum(correct_stops)
    avg_error = sum(error)/len(data)
    if(get_final_output):
        return bought, avg_error, correct_stops
    return avg_error, correct_stops

In [6]:
##Cost
best_buy_250_train = [min(ele) for ele in trajectories_train_250]
best_buy_250_test = [min(ele) for ele in trajectories_test_250]
best_buy_500_train = [min(ele) for ele in trajectories_train_500]
best_buy_500_test = [min(ele) for ele in trajectories_test_500]

##Indices
best_buy_250_train_idx = [ele.index(min(ele)) for ele in trajectories_train_250]
best_buy_250_test_idx = [ele.index(min(ele)) for ele in trajectories_test_250]
best_buy_500_train_idx = [ele.index(min(ele)) for ele in trajectories_train_500]
best_buy_500_test_idx = [ele.index(min(ele)) for ele in trajectories_test_500]

In [7]:
evaluate_policy_two_split((180, 220, 7, 0), trajectories_train_250, best_buy_250_train, best_buy_250_train_idx, (8,7), False)


(44.642, 178)

In [8]:
def policy_search_two_split(price_low_bound, price_upper_bound, data, best_buys_cost, best_buys_idx, step_size, split):
    all_errors = []
    all_correct_stops = []
    all_policies = generate_policies_two_split(price_low_bound, price_upper_bound, step_size, split)
    for idx, policy in enumerate(all_policies):
        policy_avg_error, correct_stops = evaluate_policy_two_split(policy, data, best_buys_cost, best_buys_idx, split, False)
        sys.stdout.flush()
        sys.stdout.write("\rProgress: {}/{}".format(idx+1, len(all_policies)))
        all_errors.append(policy_avg_error)
        all_correct_stops.append(correct_stops)
    optimal_policy_by_avg_error = (min(all_errors), all_policies[all_errors.index(min(all_errors))])
    optimal_policy_by_opt_stop = (max(all_correct_stops), all_policies[all_correct_stops.index(max(all_correct_stops))])
    return optimal_policy_by_avg_error, optimal_policy_by_opt_stop


# Route 1 (250)

### Split 8-7

In [35]:
optimal_policy_250 = policy_search_two_split(180, 231, trajectories_train_250, 
                                             best_buy_250_train, 
                                             best_buy_250_train_idx, 
                                             10, (8,7))
optimal_policy_250

Progress: 2016/2016

((24.737, (220, 180, 7, 0)), (341, (210, 180, 7, 0)))

In [15]:
final_result_8_7_250, avg_error, correct_stops = evaluate_policy_two_split((220, 180, 7, 0), trajectories_test_250, 
                          best_buy_250_test, 
                          best_buy_250_test_idx, 
                          (8,7), True)
avg_error, correct_stops

(26.133333333333333, 10)

### Split 10-5

In [38]:
optimal_policy_250_alt_split_1 = policy_search_two_split(180, 231, trajectories_train_250, 
                                             best_buy_250_train, 
                                             best_buy_250_train_idx, 
                                             10, (10,5))
optimal_policy_250_alt_split_1

Progress: 1800/1800

((20.57, (220, 180, 9, 0)), (401, (210, 180, 9, 0)))

In [16]:
final_result_10_5_250, avg_error, correct_stops = evaluate_policy_two_split((220, 180, 9, 0), trajectories_test_250, 
                          best_buy_250_test, 
                          best_buy_250_test_idx, 
                          (10,5), True)
avg_error, correct_stops

(23.066666666666666, 12)

### Split 5-10

In [41]:
optimal_policy_250_alt_split_2 = policy_search_two_split(180, 231, trajectories_train_250, 
                                             best_buy_250_train, 
                                             best_buy_250_train_idx, 
                                             10, (5,10))
optimal_policy_250_alt_split_2

Progress: 1800/1800

((30.328, (230, 180, 4, 0)), (241, (210, 180, 4, 0)))

In [17]:
final_result_5_10_250, avg_error, correct_stops = evaluate_policy_two_split((230, 180, 4, 0), trajectories_test_250, 
                          best_buy_250_test, 
                          best_buy_250_test_idx, 
                          (5,10), True)
avg_error, correct_stops

(24.683333333333334, 8)

# Route 2 (500)

### Split 8-7

In [54]:
optimal_policy_500_alt1 = policy_search_two_split(370, 501, trajectories_train_500, 
                                   best_buy_500_train, 
                                   best_buy_500_train_idx, 
                                   10, (8,7))
print(optimal_policy_500_alt1)


Progress: 10976/10976((46.291, (440, 370, 7, 0)), (374, (420, 370, 7, 0)))


(44.96666666666667, 15)

In [20]:
final_result_8_7_500, avg_error, correct_stops = evaluate_policy_two_split((440, 370, 7, 0), trajectories_test_500, 
                          best_buy_500_test, 
                          best_buy_500_test_idx, 
                          (8,7), True)
avg_error, correct_stops

(44.96666666666667, 15)

### Split 10-5

In [52]:
optimal_policy_500 = policy_search_two_split(370, 501, trajectories_train_500, 
                                   best_buy_500_train, 
                                   best_buy_500_train_idx, 
                                   10, (10,5))
optimal_policy_500

Progress: 9800/9800

((39.629, (430, 370, 9, 0)), (444, (420, 370, 9, 0)))

In [21]:
final_result_10_5_500, avg_error, correct_stops = evaluate_policy_two_split((430, 370, 9, 0), trajectories_test_500, 
                          best_buy_500_test, 
                          best_buy_500_test_idx, 
                          (10,5), True)
avg_error, correct_stops

(42.11666666666667, 18)

In [22]:
optimal_policy_500 = policy_search_two_split(370, 501, trajectories_train_500, 
                                   best_buy_500_train, 
                                   best_buy_500_train_idx, 
                                   10, (5,10))
optimal_policy_500

Progress: 9800/9800

((61.828, (460, 370, 4, 0)), (258, (430, 370, 4, 0)))

In [23]:
final_result_5_10_500, avg_error, correct_stops = evaluate_policy_two_split((460, 370, 4, 0), trajectories_test_500, 
                          best_buy_500_test, 
                          best_buy_500_test_idx, 
                          (5,10), True)
avg_error, correct_stops

(50.13333333333333, 10)

In [32]:
rl_two_split_8_7 = [final_result_8_7_250 + final_result_8_7_500]
rl_two_split_5_10 = [final_result_5_10_250 + final_result_5_10_500]
rl_two_split_10_5 = [final_result_10_5_250 + final_result_10_5_500]

In [33]:
def save_data(obj, filename):
    final_file_object = open(filename, 'wb')
    pickle.dump(obj, final_file_object)
    final_file_object.close()

In [35]:
save_data(rl_two_split_8_7, "../Human Experiments/Tests/rl_two_split_8_7")
save_data(rl_two_split_5_10, "../Human Experiments/Tests/rl_two_split_5_10")
save_data(rl_two_split_10_5, "../Human Experiments/Tests/rl_two_split_10_5")